# Group Lab 3 – Comparative Urban Change in US States

Authors: Andrew Baker, Jamie Marken, James Lyou, Alberto Melendez, Emmanual Robi

## Part 2

In [1]:
#Import pandas and geopandas
import pandas as pd
import geopandas

In [2]:
#Read the SAEP data
fp = "./saep_bg10/saep_bg10.shp"
data = geopandas.read_file(fp)

In [3]:
#Read the FIPS dbf data
fp2 = "./WashingtonFIPS.dbf"
dbf = geopandas.read_file(fp2)

In [ ]:
#Divides SAEP data into counties and write the individual divisions to JSON files
county_counter = 0
for row in dbf.FIPSCounty:
    is_in_county = data['COUNTYFP10'] == row
    subset = data[is_in_county]
    subset.to_file(dbf.CountyName[county_counter] + '.json', driver = 'GeoJSON')
    county_counter += 1

In [ ]:
#Creates a new dataframe with total populations of each county in 2017
#and prints the top 10 most populous counties
county_pop = pd.DataFrame()
county_pop['FIPSCounty'] = dbf.FIPSCounty
county_pop['CountyName'] = dbf.CountyName
county_pop['Population'] = 0
counter = 0
    
for row in county_pop.FIPSCounty:
    is_in_county = data['COUNTYFP10'] == row
    subset = data[is_in_county]
    county_pop.Population[counter] = sum(subset['POP2017'] + county_pop.Population[counter])
    counter += 1

county_pop = county_pop.sort_values('Population', ascending=False)
print(county_pop.head(n=10))

## Part 3

In [ ]:
#imports necessary packages
import geopandas as gpd
import numpy as np

In [ ]:
fp = "./saep_bg10/saep_bg10.shp"
data = gpd.read_file(fp)

#### Part 3.1

In [ ]:
#calculates density to determine if a group is urban or non-urban (1000 people in 1 sq mile is urban)
data["urban2018"] = np.where((data['POP2018'] / data['ALANDMI'])>= 1000, 'urban', 'non-urban')

#### Part 3.2

In [ ]:
urban_counter = 0
for row in data['urban2018']:
    if row == 'urban':
        urban_counter += 1

In [ ]:
#calculates percentage of groupds that are urban in washington
percent_State_Urban = (float(urban_counter)/4783) * 100
print percent_State_Urban

#### Part 3.3

In [ ]:
urban_land = data

urban_land = urban_land.drop(urban_land[urban_land["urban2018"] != "urban"].index)

In [ ]:
land_sum = data['ALANDMI'].sum()
urban_land_sum = urban_land['ALANDMI'].sum()

In [ ]:
percent_urbanized = (urban_land_sum / land_sum) * 100
print percent_urbanized

#### Part 3.4

In [ ]:
#calculates density of population
data["urban2008"] = np.where((data["POP2008"]/data["ALANDMI"]) >= 1000, 'urban', 'non-urban')

In [ ]:
#initializes a column and then compares columns to determine if there is a change
data['ClassChange'] = ''
data["ClassChange"] = np.where((data['urban2008'] == data['urban2018']), 'no change in category', data['ClassChange'])

In [ ]:
#keywords are used similarly like the code above, determines change but had to be done slightly differently
keyword1 = 'urban'
keyword2 = 'non-urban'

data["ClassChange"] = np.where((data['urban2008'] == keyword2) & (data['urban2018'] == keyword1), 'urbanized', data['ClassChange'])
data["ClassChange"] = np.where((data['urban2008'] == keyword1) & (data['urban2018'] == keyword2), 'de-urbanized', data['ClassChange'])

In [ ]:
data

#### Part 3.5

In [ ]:
print data['ClassChange'].value_counts()

#### Part 3.6

In [ ]:
%matplotlib notebook

#removes non-populated blocks (mostly ones that are water)
mapdata = data.drop(data[data['POP2018'] == 0].index)
mapdata.plot(column = "ClassChange",figsize= (10,8), legend = True)

In [ ]:
%matplotlib notebook
mapdata.plot(column = "urban2018",figsize= (10,8), legend = True)